In [1]:
import pandas as pd
import numpy as np

In [22]:
df = pd.read_html('DetailedStatement.htm')

In [23]:
df = df[0]
print(df)

                        0                     1                            2   \
0      Account: 1100013128   Account: 1100013128          Name: zhong zhenhai   
1     Closed Transactions:  Closed Transactions:         Closed Transactions:   
2                   Ticket             Open Time                         Type   
3                111293032   2023.12.29 14:32:53                         sell   
4                111271130   2023.12.28 16:42:23                          buy   
...                    ...                   ...                          ...   
1716               Largest               Largest                profit trade:   
1717               Average               Average                profit trade:   
1718               Maximum               Maximum        consecutive wins ($):   
1719               Maximal               Maximal  consecutive profit (count):   
1720               Average               Average            consecutive wins:   

                           

In [24]:
 col_list = df.loc[2].values.tolist()

In [25]:
col_list

['Ticket',
 'Open Time',
 'Type',
 'Size',
 'Item',
 'Price',
 'S / L',
 'T / P',
 'Close Time',
 'Price',
 'Commission',
 'Taxes',
 'Swap',
 'Profit']

In [26]:
df.columns = col_list

In [27]:
df.drop([0,1,2],inplace = True)

In [28]:
trade_history = pd.concat([df[ df['Type'] == 'sell'],df[ df['Type'] == 'buy']])

In [29]:
trade_history = trade_history[trade_history['Commission'] != '0.00']

In [30]:
xauusd = pd.read_csv('XAUUSD_M1.csv','\t')

In [31]:
trade_history['<DATE>'] = trade_history['Open Time'].str.slice(0,10)
trade_history['<TIME>'] = trade_history['Open Time'].str.slice(11,17)+'00'

In [32]:
ma_list = [34,170,850,2040]
for ma in ma_list:
    xauusd['ma'+str(ma)] = xauusd['<CLOSE>'].rolling(ma,center = False).mean()
xauusd.sort_values(by = ['<DATE>','<TIME>'],ascending = False)

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>,ma34,ma170,ma850,ma2040
689447,2023.12.15,23:54:00,2018.60,2018.60,2018.09,2018.39,143,0,30,2018.959706,2018.517000,2033.179200,2034.770686
689446,2023.12.15,23:53:00,2018.52,2018.62,2018.24,2018.60,83,0,30,2018.987941,2018.520471,2033.202906,2034.778652
689445,2023.12.15,23:52:00,2018.41,2018.65,2018.26,2018.54,124,0,29,2019.010882,2018.525118,2033.226247,2034.786721
689444,2023.12.15,23:51:00,2018.57,2018.57,2018.30,2018.40,36,0,30,2019.031765,2018.533765,2033.249541,2034.794907
689443,2023.12.15,23:50:00,2018.28,2018.57,2018.25,2018.48,61,0,30,2019.045882,2018.539824,2033.272541,2034.803152
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022.01.03,01:09:00,1831.13,1831.82,1830.86,1831.65,58,0,36,NaN,NaN,NaN,NaN
3,2022.01.03,01:08:00,1829.87,1831.13,1829.80,1831.13,45,0,42,NaN,NaN,NaN,NaN
2,2022.01.03,01:07:00,1829.90,1829.95,1829.52,1829.79,47,0,52,NaN,NaN,NaN,NaN
1,2022.01.03,01:06:00,1829.75,1830.16,1829.65,1830.02,56,0,57,NaN,NaN,NaN,NaN


In [33]:
xauusd['should'] = np.where((xauusd['ma34'] > xauusd['<HIGH>']) &(xauusd['ma170'] > xauusd['<HIGH>']),'sell','')

In [34]:
xauusd['should'] = np.where((xauusd['ma34'] < xauusd['<LOW>']) &(xauusd['ma170'] < xauusd['<LOW>']),'buy',xauusd['should'])

In [35]:
xauusd.groupby(xauusd['should']).count()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>,ma34,ma170,ma850,ma2040
should,,,,,,,,,,,,,
,290533,290533,290533,290533,290533,290533,290533,290533,290533,290500,290364,290120,289665
buy,197886,197886,197886,197886,197886,197886,197886,197886,197886,197886,197886,197773,197408
sell,201029,201029,201029,201029,201029,201029,201029,201029,201029,201029,201029,200706,200336


In [36]:
xauusd

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>,ma34,ma170,ma850,ma2040,should
0,2022.01.03,01:05:00,1830.63,1830.64,1829.65,1829.73,45,0,50,NaN,NaN,NaN,NaN,
1,2022.01.03,01:06:00,1829.75,1830.16,1829.65,1830.02,56,0,57,NaN,NaN,NaN,NaN,
2,2022.01.03,01:07:00,1829.90,1829.95,1829.52,1829.79,47,0,52,NaN,NaN,NaN,NaN,
3,2022.01.03,01:08:00,1829.87,1831.13,1829.80,1831.13,45,0,42,NaN,NaN,NaN,NaN,
4,2022.01.03,01:09:00,1831.13,1831.82,1830.86,1831.65,58,0,36,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689443,2023.12.15,23:50:00,2018.28,2018.57,2018.25,2018.48,61,0,30,2019.045882,2018.539824,2033.272541,2034.803152,
689444,2023.12.15,23:51:00,2018.57,2018.57,2018.30,2018.40,36,0,30,2019.031765,2018.533765,2033.249541,2034.794907,
689445,2023.12.15,23:52:00,2018.41,2018.65,2018.26,2018.54,124,0,29,2019.010882,2018.525118,2033.226247,2034.786721,
689446,2023.12.15,23:53:00,2018.52,2018.62,2018.24,2018.60,83,0,30,2018.987941,2018.520471,2033.202906,2034.778652,


In [37]:
trade_history['Profit'] = trade_history['Profit'].str.replace(' ','').astype('float64')

In [38]:
trade_history

,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price,Commission,Taxes,Swap,Profit,<DATE>,<TIME>
3,111293032,2023.12.29 14:32:53,sell,0.30,xauusd,2066.45,2067.52,2063.24,2023.12.29 15:07:28,2063.21,-2.10,0.00,0.00,97.2,2023.12.29,14:32:00
6,111237904,2023.12.27 13:20:34,sell,0.30,xauusd,2067.18,2068.30,2066.29,2023.12.27 13:50:30,2067.92,-2.10,0.00,0.00,-22.2,2023.12.27,13:20:00
8,111192832,2023.12.22 13:43:06,sell,1.00,xauusd,2061.06,2063.37,2058.28,2023.12.22 13:52:44,2058.23,-7.00,0.00,0.00,283.0,2023.12.22,13:43:00
10,111173490,2023.12.21 17:06:55,sell,0.30,xauusd,2038.50,0.00,0.00,2023.12.21 17:10:07,2038.29,-2.10,0.00,0.00,6.3,2023.12.21,17:06:00
23,111042750,2023.12.14 16:45:17,sell,0.30,xauusd,2047.74,2047.50,2042.94,2023.12.14 16:50:06,2043.10,-2.10,0.00,0.00,139.2,2023.12.14,16:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,85477604,2022.03.01 15:23:16,buy,0.20,xauusd,1919.32,1915.97,1919.10,2022.03.01 15:29:38,1919.15,-1.40,0.00,0.00,-3.4,2022.03.01,15:23:00
1680,85477005,2022.03.01 15:00:00,buy,0.20,xauusd,1916.63,1913.07,0.00,2022.03.01 15:28:08,1918.04,-1.40,0.00,0.00,28.2,2022.03.01,15:00:00
1684,85474590,2022.03.01 12:35:34,buy,1.00,xauusd,1921.33,1920.34,1924.69,2022.03.01 12:48:43,1923.00,-7.00,0.00,0.00,167.0,2022.03.01,12:35:00
1688,85473694,2022.03.01 11:49:00,buy,0.20,xauusd,1915.93,1915.14,0.00,2022.03.01 12:05:03,1916.81,-1.40,0.00,0.00,17.6,2022.03.01,11:49:00


In [39]:
res = pd.merge(xauusd,trade_history,how = 'right',on = ['<DATE>','<TIME>'])

In [40]:
res

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>,ma34,...,Item,Price,S / L,T / P,Close Time,Price,Commission,Taxes,Swap,Profit
0,2023.12.29,14:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,xauusd,2066.45,2067.52,2063.24,2023.12.29 15:07:28,2063.21,-2.10,0.00,0.00,97.2
1,2023.12.27,13:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,xauusd,2067.18,2068.30,2066.29,2023.12.27 13:50:30,2067.92,-2.10,0.00,0.00,-22.2
2,2023.12.22,13:43:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,xauusd,2061.06,2063.37,2058.28,2023.12.22 13:52:44,2058.23,-7.00,0.00,0.00,283.0
3,2023.12.21,17:06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,xauusd,2038.50,0.00,0.00,2023.12.21 17:10:07,2038.29,-2.10,0.00,0.00,6.3
4,2023.12.14,16:45:00,2047.80,2047.90,2046.07,2046.53,94.0,0.0,6.0,2040.585588,...,xauusd,2047.74,2047.50,2042.94,2023.12.14 16:50:06,2043.10,-2.10,0.00,0.00,139.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,2022.03.01,15:23:00,1919.71,1920.01,1917.36,1917.44,217.0,0.0,5.0,1918.449118,...,xauusd,1919.32,1915.97,1919.10,2022.03.01 15:29:38,1919.15,-1.40,0.00,0.00,-3.4
1679,2022.03.01,15:00:00,1916.36,1916.88,1914.98,1915.13,240.0,0.0,5.0,1920.442941,...,xauusd,1916.63,1913.07,0.00,2022.03.01 15:28:08,1918.04,-1.40,0.00,0.00,28.2
1680,2022.03.01,12:35:00,1921.60,1921.93,1920.81,1921.49,187.0,0.0,5.0,1918.030588,...,xauusd,1921.33,1920.34,1924.69,2022.03.01 12:48:43,1923.00,-7.00,0.00,0.00,167.0
1681,2022.03.01,11:49:00,1915.72,1915.86,1915.05,1915.05,153.0,0.0,5.0,1915.927647,...,xauusd,1915.93,1915.14,0.00,2022.03.01 12:05:03,1916.81,-1.40,0.00,0.00,17.6


In [41]:
res['trade'] = np.where(res['Type'] == res['should'] ,'yes','no')

In [42]:
res['result'] = np.where(res['Profit'] >= 0 ,'yes','no')

In [44]:
res.groupby([res['trade'],res['result']]).mean()

<OPEN>       <HIGH>        <LOW>      <CLOSE>   <TICKVOL>  \
trade result                                                                   
no    no      1812.723324  1813.252816  1812.123736  1812.668874  193.054945   
      yes     1825.970962  1826.612464  1825.424009  1826.067522  191.749271   
yes   no      1818.849160  1819.350504  1818.240924  1818.814538  184.058824   
      yes     1822.799403  1823.351866  1822.149104  1822.677463  210.276119   

              <VOL>  <SPREAD>         ma34        ma170        ma850  \
trade result                                                           
no    no        0.0  4.839286  1813.475555  1813.310948  1814.367291   
      yes       0.0  4.775510  1826.506851  1826.564431  1827.531474   
yes   no        0.0  4.478992  1818.445393  1818.198860  1819.156984   
      yes       0.0  4.716418  1821.015215  1819.739481  1819.297503   

                   ma2040     Profit  
trade result                          
no    no      1815.013649 -72.714152  
      yes     1827.905890  72.381934  
yes   no      1821.161135 -41.838319  
      yes     1820.206996  38.617388

In [45]:
res.groupby(res['trade']).count()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>,ma34,...,S / L,T / P,Close Time,Price,Commission,Taxes,Swap,Profit,trade,result
trade,,,,,,,,,,,,,,,,,,,,,
no,1430,1430,1414,1414,1414,1414,1414,1414,1414,1414,...,1430,1430,1430,1430,1430,1430,1430,1430,1430,1430
yes,253,253,253,253,253,253,253,253,253,253,...,253,253,253,253,253,253,253,253,253,253


In [46]:
res.mean()

<OPEN>       1819.422214
<HIGH>       1819.997636
<LOW>        1818.839592
<CLOSE>      1819.425897
<TICKVOL>     193.259748
<VOL>           0.000000
<SPREAD>        4.777445
ma34         1819.799007
ma170        1819.630669
ma850        1820.522811
ma2040       1821.175338
Ticket               inf
Profit         -1.921378
dtype: float64

In [47]:
res['year'] = res['<DATE>'].str.slice(0,4)
res['mon'] = res['<DATE>'].str.slice(5,7)
res['day'] = res['<DATE>'].str.slice(8,10)



In [48]:
res['Commission'] = res['Commission'].str.replace(' ','').astype('float64')

In [49]:
res['Open Time'] = res['Open Time'].astype('datetime64[ns]')


In [50]:
res.sort_values(by = 'Open Time',ascending = False, inplace = True)

In [51]:
res.columns

Index(['<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<TICKVOL>',
       '<VOL>', '<SPREAD>', 'ma34', 'ma170', 'ma850', 'ma2040', 'should',
       'Ticket', 'Open Time', 'Type', 'Size', 'Item', 'Price', 'S / L',
       'T / P', 'Close Time', 'Price', 'Commission', 'Taxes', 'Swap', 'Profit',
       'trade', 'result', 'year', 'mon', 'day'],
      dtype='object')

In [52]:
pivot_table = pd.pivot_table(res,index = ['year', 'mon', 'day','Open Time'],values = ['Commission','Profit'],aggfunc = 'sum' ,margins = 'True')

In [36]:
# out = pd.concat([d.append(d.sum().rename((k, '', '', '', 'Subtotal'))) for k, d in table.groupby('Region')]).append(table.sum().rename(('Grand', '', '', '', 'Total')))

# out.index = pd.MultiIndex.from_tuples(out.index)

In [53]:
def get_subtotal(table, sub_total='subtotal', get_total=False, total='TOTAL'):
    """
    Parameters
    ----------
    table : dataframe, table with multi-index resulting from pd.pivot_table() or 
    df.groupby().
    sub_total : str, optional
        Name given to the subtotal. The default is '_Sous-total'.
    get_total : boolean, optional
        Precise if you want to add the final total (in case you used groupeby()). 
        The default is False.
    total : str, optional
        Name given to the total. The default is 'TOTAL'.

    Returns
    -------
    A table with the total and subtotal added.
    """
    index_name1 = table.index.names[0]
    index_name2 = table.index.names[1]
    
    pvt = table.unstack(0)
    mask = pvt.columns.get_level_values(index_name1) != 'All'
    #print (mask)
    pvt.loc[sub_total] = pvt.loc[:, mask].sum()
    
    pvt = pvt.stack().swaplevel(0,1).sort_index()
    pvt = pvt[pvt.columns[1:].tolist() + pvt.columns[:1].tolist()]
    
    if get_total:
        mask = pvt.index.get_level_values(index_name2) != sub_total
        pvt.loc[(total, '' ),: ] = pvt.loc[mask].sum()
    print (pvt)
    return(pvt)

In [54]:
new_table = get_subtotal(pivot_table)

                                     Profit  Commission
year                                                   
2022 (03, 01, 2022-03-01 11:39:10)    -5.40       -1.40
     (03, 01, 2022-03-01 11:42:45)    20.80       -1.40
     (03, 01, 2022-03-01 11:49:00)    17.60       -1.40
     (03, 01, 2022-03-01 12:11:04)   -44.40       -1.40
     (03, 01, 2022-03-01 12:31:03)    -9.20       -1.40
...                                     ...         ...
2023 (12, 27, 2023-12-27 13:50:34)    11.00       -7.00
     (12, 28, 2023-12-28 16:42:23)   103.20       -2.10
     (12, 29, 2023-12-29 14:32:53)    97.20       -2.10
     subtotal                        536.00     -257.04
All  (, , NaT)                     -3233.68    -4593.19

[1686 rows x 2 columns]


In [55]:
unstack_table = pivot_table.unstack([0,1,2])

In [56]:
unstack_table

Commission                                      ...  \
year                      2022                                      ...   
mon                         03                                      ...   
day                         01  02  03  04  07  08  09  10  11  14  ...   
Open Time                                                           ...   
NaT                        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   
2022-03-01 11:39:10       -1.4 NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   
2022-03-01 11:42:45       -1.4 NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   
2022-03-01 11:49:00       -1.4 NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   
2022-03-01 12:11:04       -1.4 NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   
...                        ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...   
2023-12-22 15:37:42        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   
2023-12-27 13:20:34        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   
2023-12-27 13:50:34        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   
2023-12-28 16:42:23        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   
2023-12-29 14:32:53        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   

                    Profit                                                     
year                  2023                                                All  
mon                     12                                                     
day                     05  12  14  15  21     22    27     28    29           
Open Time                                                                      
NaT                    NaN NaN NaN NaN NaN    NaN   NaN    NaN   NaN -3233.68  
2022-03-01 11:39:10    NaN NaN NaN NaN NaN    NaN   NaN    NaN   NaN      NaN  
2022-03-01 11:42:45    NaN NaN NaN NaN NaN    NaN   NaN    NaN   NaN      NaN  
2022-03-01 11:49:00    NaN NaN NaN NaN NaN    NaN   NaN    NaN   NaN      NaN  
2022-03-01 12:11:04    NaN NaN NaN NaN NaN    NaN   NaN    NaN   NaN      NaN  
...                    ...  ..  ..  ..  ..    ...   ...    ...   ...      ...  
2023-12-22 15:37:42    NaN NaN NaN NaN NaN  573.0   NaN    NaN   NaN      NaN  
2023-12-27 13:20:34    NaN NaN NaN NaN NaN    NaN -22.2    NaN   NaN      NaN  
2023-12-27 13:50:34    NaN NaN NaN NaN NaN    NaN  11.0    NaN   NaN      NaN  
2023-12-28 16:42:23    NaN NaN NaN NaN NaN    NaN   NaN  103.2   NaN      NaN  
2023-12-29 14:32:53    NaN NaN NaN NaN NaN    NaN   NaN    NaN  97.2      NaN  

[1684 rows x 408 columns]

In [57]:
mask = unstack_table.columns.get_level_values('day') != 'All'

In [58]:
mask

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [59]:
unstack_table.loc['subtotal'] = unstack_table.loc[:, mask].sum()

In [60]:
pivot_table = unstack_table.stack([1,2,3]).swaplevel(0,1).sort_index(level=[0,2,3])

In [61]:
pivot_table

Commission   Profit
year Open Time           mon day                     
2022 2022-03-01 11:39:10 03  01        -1.40    -5.40
     2022-03-01 11:42:45 03  01        -1.40    20.80
     2022-03-01 11:49:00 03  01        -1.40    17.60
     2022-03-01 12:11:04 03  01        -1.40   -44.40
     2022-03-01 12:31:03 03  01        -1.40    -9.20
...                                      ...      ...
2023 subtotal            12  28        -2.10   103.20
     2023-12-29 14:32:53 12  29        -2.10    97.20
     subtotal            12  29        -2.10    97.20
All  NaN                            -4593.19 -3233.68
     subtotal                       -4593.19 -3233.68

[1888 rows x 2 columns]

In [62]:
pivot_table = pivot_table[pivot_table.columns[1:].tolist() + pivot_table.columns[:1].tolist()]

In [63]:
pivot_table

Profit  Commission
year Open Time           mon day                     
2022 2022-03-01 11:39:10 03  01     -5.40       -1.40
     2022-03-01 11:42:45 03  01     20.80       -1.40
     2022-03-01 11:49:00 03  01     17.60       -1.40
     2022-03-01 12:11:04 03  01    -44.40       -1.40
     2022-03-01 12:31:03 03  01     -9.20       -1.40
...                                   ...         ...
2023 subtotal            12  28    103.20       -2.10
     2023-12-29 14:32:53 12  29     97.20       -2.10
     subtotal            12  29     97.20       -2.10
All  NaN                         -3233.68    -4593.19
     subtotal                    -3233.68    -4593.19

[1888 rows x 2 columns]

In [67]:
with pd.ExcelWriter('result.xlsx', mode = 'a+') as writer:
    res.to_excel(writer, sheet_name = 'res')
    pivot_table.to_excel(writer, sheet_name = 'pivot_table')
    new_table.to_excel(writer, sheet_name = 'new_table')

In [68]:
dict_data = [{'duration': 0.7, 'project_id': 3, 'resource': u'Arya Stark', 'activity': u'Development'},
{'duration': 0.9, 'project_id': 4, 'resource': u'Ned Stark', 'activity': u'Development'},
{'duration': 2.88, 'project_id': 7, 'resource': u'Robb Stark', 'activity': u'Development'},
{'duration': 0.22, 'project_id': 9, 'resource': u'Jon Snow', 'activity': u'Support'},
{'duration': 0.3, 'project_id': 9, 'resource': u'Jon Snow', 'activity': u'Support'},
{'duration': 2.15, 'project_id': 3, 'resource': u'Arya Stark', 'activity': u'Practise'},
{'duration': 3.35, 'project_id': 4, 'resource': u'Sansa Stark', 'activity': u'Development'},
{'duration': 2.17, 'project_id': 9, 'resource': u'Rickon Stark', 'activity': u'Development'},
{'duration': 1.03, 'project_id': 4, 'resource': u'Benjan Stark', 'activity': u'Design'},
{'duration': 1.77, 'project_id': 4, 'resource': u'Bran Stark', 'activity': u'Testing'},
{'duration': 1.17, 'project_id': 4, 'resource': u'Ned Stark', 'activity': u'Development'},
{'duration': 0.17, 'project_id': 9, 'resource': u'Jon Snow', 'activity': u'Support'},
{'duration': 1.77, 'project_id': 3, 'resource': u'catelyn stark', 'activity': u'Development'},
{'duration': 0.3, 'project_id': 9, 'resource': u'Jon Snow', 'activity': u'Support'},
{'duration': 0.45, 'project_id': 9, 'resource': u'Jon Snow', 'activity': u'Support'}]

In [69]:
df = pd.DataFrame(dict_data)

In [71]:
pvt = pd.pivot_table(df, values=['duration'],index=['project_id','resource'], columns=['activity'], aggfunc=np.sum,margins=True, fill_value=0)

In [72]:
pvt

duration                                            
activity                   Design Development Practise Support Testing    All
project_id resource                                                          
3          Arya Stark        0.00        0.70     2.15    0.00    0.00   2.85
           catelyn stark     0.00        1.77     0.00    0.00    0.00   1.77
4          Benjan Stark      1.03        0.00     0.00    0.00    0.00   1.03
           Bran Stark        0.00        0.00     0.00    0.00    1.77   1.77
           Ned Stark         0.00        2.07     0.00    0.00    0.00   2.07
           Sansa Stark       0.00        3.35     0.00    0.00    0.00   3.35
7          Robb Stark        0.00        2.88     0.00    0.00    0.00   2.88
9          Jon Snow          0.00        0.00     0.00    1.44    0.00   1.44
           Rickon Stark      0.00        2.17     0.00    0.00    0.00   2.17
All                          1.03       12.94     2.15    1.44    1.77  19.33

In [73]:
pvt = pvt.unstack(0)


In [74]:
pvt

duration                                                      \
activity        Design                       Development                     
project_id           3     4    7    9   All           3     4     7     9   
resource                                                                     
                   NaN   NaN  NaN  NaN  1.03         NaN   NaN   NaN   NaN   
Arya Stark         0.0   NaN  NaN  NaN   NaN        0.70   NaN   NaN   NaN   
Benjan Stark       NaN  1.03  NaN  NaN   NaN         NaN  0.00   NaN   NaN   
Bran Stark         NaN  0.00  NaN  NaN   NaN         NaN  0.00   NaN   NaN   
Jon Snow           NaN   NaN  NaN  0.0   NaN         NaN   NaN   NaN  0.00   
Ned Stark          NaN  0.00  NaN  NaN   NaN         NaN  2.07   NaN   NaN   
Rickon Stark       NaN   NaN  NaN  0.0   NaN         NaN   NaN   NaN  2.17   
Robb Stark         NaN   NaN  0.0  NaN   NaN         NaN   NaN  2.88   NaN   
Sansa Stark        NaN  0.00  NaN  NaN   NaN         NaN  3.35   NaN   NaN   
catelyn stark      0.0   NaN  NaN  NaN   NaN        1.77   NaN   NaN   NaN   

                      ...                                                  \
activity              ... Testing                         All               
project_id       All  ...       3     4    7    9   All     3     4     7   
resource              ...                                                   
               12.94  ...     NaN   NaN  NaN  NaN  1.77   NaN   NaN   NaN   
Arya Stark       NaN  ...     0.0   NaN  NaN  NaN   NaN  2.85   NaN   NaN   
Benjan Stark     NaN  ...     NaN  0.00  NaN  NaN   NaN   NaN  1.03   NaN   
Bran Stark       NaN  ...     NaN  1.77  NaN  NaN   NaN   NaN  1.77   NaN   
Jon Snow         NaN  ...     NaN   NaN  NaN  0.0   NaN   NaN   NaN   NaN   
Ned Stark        NaN  ...     NaN  0.00  NaN  NaN   NaN   NaN  2.07   NaN   
Rickon Stark     NaN  ...     NaN   NaN  NaN  0.0   NaN   NaN   NaN   NaN   
Robb Stark       NaN  ...     NaN   NaN  0.0  NaN   NaN   NaN   NaN  2.88   
Sansa Stark      NaN  ...     NaN  0.00  NaN  NaN   NaN   NaN  3.35   NaN   
catelyn stark    NaN  ...     0.0   NaN  NaN  NaN   NaN  1.77   NaN   NaN   

                            
activity                    
project_id        9    All  
resource                    
                NaN  19.33  
Arya Stark      NaN    NaN  
Benjan Stark    NaN    NaN  
Bran Stark      NaN    NaN  
Jon Snow       1.44    NaN  
Ned Stark       NaN    NaN  
Rickon Stark   2.17    NaN  
Robb Stark      NaN    NaN  
Sansa Stark     NaN    NaN  
catelyn stark   NaN    NaN  

[10 rows x 30 columns]

In [44]:
mask = pvt.columns.get_level_values('project_id') != 'All'
mask

array([ True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True, False])

In [ ]:
pvt.loc['subtotal'] = pvt.loc[:, mask].sum()

In [ ]:
pvt

In [ ]:
pvt = pvt.stack().swaplevel(0,1).sort_index()

In [ ]:
pvt = pvt[pvt.columns[1:].tolist() + pvt.columns[:1].tolist()]

In [ ]:
pvt